In [1]:
import random
import numpy as np
import torch
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [2]:

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re



def load_label_file(path: str) -> str:
    """key: value1,value2,... 형식으로 된 .txt 파일을 통째로 읽어서 문자열로 반환"""
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

def parse_key_value_lines(text: str):
    """'key:val1,val2,...' 여러 줄을 딕셔너리로 변환"""
    id2label = {}
    for line in text.splitlines():
        line = line.strip()
        if not line or ":" not in line:
            continue
        key, vals = line.split(":", 1)
        id2label[key.strip()] = vals.strip()
    return id2label

def preprocess_label_text(label_path_str: str):
    cleaned = label_path_str.lower()
    cleaned = re.sub(r"[:,]", " ", cleaned)
    cleaned = re.sub(r"_", " ", cleaned)
    cleaned = re.sub(r"[^a-z0-9 ]", " ", cleaned)
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    return cleaned

def build_tfidf_vectorizer(label_texts):
    vectorizer = TfidfVectorizer()
    label_tfidf = vectorizer.fit_transform(label_texts)
    return vectorizer, label_tfidf

def compute_lexical_similarity(doc_text, vectorizer, label_tfidf):
    doc_vec = vectorizer.transform([doc_text])
    sims = cosine_similarity(doc_vec, label_tfidf)[0]
    return sims

    
label_raw_text = load_label_file("Amazon_products/class_related_keywords.txt")  # 네 파일 이름에 맞춰 바꿔
id2label = parse_key_value_lines(label_raw_text)

# 2) 라벨 텍스트 전처리해서 TF-IDF 학습
label_keys = list(id2label.keys())
label_texts = [
    preprocess_label_text(f"{k} {id2label[k]}")
    for k in label_keys
]
vectorizer, label_tfidf = build_tfidf_vectorizer(label_texts)

# 3) 테스트용 문서 하나 넣어보기
doc = "gourmet organic chocolate snack"
doc_clean = preprocess_label_text(doc)
sims = compute_lexical_similarity(doc_clean, vectorizer, label_tfidf)

# 4) 결과 보기
label_sims = list(zip(label_keys, sims))
label_sims.sort(key=lambda x: x[1], reverse=True)

for lbl, score in label_sims:
    print(lbl, round(score, 4))


chocolate_bars 0.5059
chocolate_gifts 0.4276
chocolate 0.3754
chocolate_covered_fruit 0.3548
dried_fruit_raisins 0.2765
chocolate_pretzels 0.2373
fresh_baked_cookies 0.2332
grocery_gourmet_food 0.2266
snack_gifts 0.2254
chocolate_assortments 0.2173
candy_chocolate 0.1635
hot_cocoa 0.1609
food 0.1457
gourmet_gifts 0.1342
snack_food 0.121
trail_mix 0.1181
granola_trail_mix_bars 0.1044
fruit_leather 0.1023
toaster_pastries 0.1015
cookies 0.0996
fruit 0.0941
raisins 0.0925
meat_poultry 0.0903
marshmallows 0.087
changing_table_pads_covers 0.0863
popcorn 0.085
granola_bars 0.0846
produce 0.0833
solid_feeding 0.0831
milk 0.0827
chocolate_truffles 0.0822
rice_cakes 0.0777
nutrition_wellness 0.0746
party_mix 0.0734
p_t_s 0.0716
fruit_gifts 0.0684
sensual_delights 0.0649
foie_gras_p_t_s 0.062
sugars_sweeteners 0.0604
salsas 0.0594
eggs 0.059
cakes 0.0569
nutrition_bars_drinks 0.0569
chocolate_covered_nuts 0.0565
dessert_gifts 0.0551
spices_gifts 0.0545
meat_gifts 0.0541
crackers 0.0533
juices 0.

In [3]:
def build_label_embeddings(label_keys, label_tfidf, dense: bool = True):
    """
    label_keys: 라벨 이름 리스트 (vectorize할 때 썼던 순서와 같아야 함)
    label_tfidf: shape = (n_labels, vocab_size) 인 sparse matrix
    dense: True면 numpy array로 바꿔서 돌려줌

    return:
        dict: {label_name: embedding_vector}
    """
    embeddings = {}
    if dense:
        label_tfidf_dense = label_tfidf.toarray()
        for i, label in enumerate(label_keys):
            embeddings[label] = label_tfidf_dense[i]
    else:
        # sparse 그대로
        for i, label in enumerate(label_keys):
            embeddings[label] = label_tfidf[i]
    return embeddings

label_embeddings = build_label_embeddings(label_keys, label_tfidf, dense=True)
print(label_embeddings["grocery_gourmet_food"].shape)  # (vocab_size,)


(3466,)


In [4]:

def load_edges(path):
    edges = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            raw = line.strip()
            if not raw or raw.startswith("#"):
                continue
            parts = raw.split()
            if len(parts) < 2:
                continue
            try:
                u, v = int(parts[0]), int(parts[1])
            except ValueError:
                continue
            edges.append((u, v))
    return edges

def find_roots(edges):
    parents = set()
    children = set()
    for u, v in edges:
        parents.add(u)
        children.add(v)
    # 부모로만 나온 애들 = 루트들
    roots = parents - children
    return sorted(roots)

# --- 사용 ---
E = load_edges("Amazon_products/class_hierarchy.txt")

N = 531
A = np.zeros((N, N), dtype=np.uint8)
for u, v in E:
    A[u, v] = 1
    A[v, u] = 1   # 탐색용으로는 무방향 인접행렬 써도 됨

B = np.zeros((N, N), dtype=np.uint8)
for u, v in E:
    B[u, v] = 1

roots = find_roots(E)
print("roots:", roots)


roots: [0, 3, 10, 23, 40, 169]


In [5]:
import pandas as pd

In [6]:
def load_docs_txt(path):
    """
    'idx<TAB>text' 형태의 파일을 읽어서
    ids: [int, ...]
    texts: [str, ...]
    을 리턴
    """
    ids = []
    texts = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            # 탭 기준
            idx_str, txt = line.split("\t", 1)
            ids.append(int(idx_str))
            texts.append(txt)
    return ids, texts



def build_doc_embeddings_from_existing_vectorizer(doc_texts, vectorizer):
    """
    doc_texts: 전처리 전의 원문 리스트
    vectorizer: 라벨에 대해 fit되어 있는 TfidfVectorizer
    return: dense numpy array [N_docs, vocab]
    """
    # 라벨이랑 동일 규칙으로 전처리
    cleaned_docs = [preprocess_label_text(t) for t in doc_texts]
    doc_tfidf = vectorizer.transform(cleaned_docs)   # sparse
    doc_emb = doc_tfidf.toarray().astype(np.float32)
    return doc_emb

# 사용 예시
# 1) 문서 읽기
doc_ids, doc_texts = load_docs_txt("Amazon_products/train/train_corpus.txt")

# 2) 라벨 때 만든 vectorizer 재사용해서 임베딩 만들기
doc_embeddings = build_doc_embeddings_from_existing_vectorizer(doc_texts, vectorizer)

In [7]:
print(doc_embeddings.shape)

(29487, 3466)


In [11]:
import numpy as np
from typing import List

def build_parents_children(adj: np.ndarray):
    C = adj.shape[0]
    parents = [np.flatnonzero(adj[:, j]).astype(np.int64) for j in range(C)]
    children = [np.flatnonzero(adj[j]).astype(np.int64) for j in range(C)]
    return parents, children

def build_siblings(parents, children):
    C = len(parents)
    sibs = [set() for _ in range(C)]
    for c in range(C):
        for p in parents[c]:
            for ch in children[p]:
                if ch != c:
                    sibs[c].add(int(ch))
    sibs = [np.array(sorted(s), dtype=np.int64) for s in sibs]
    return sibs

# --------------------------------
# 한 문서에서 conf(D, c) 계산 (candidate만)
# --------------------------------
def compute_conf_doc(
    sims: np.ndarray,
    cand: np.ndarray,
    parents,
    siblings,
) -> np.ndarray:
    """
    sims : [C]  (sim(D, c))
    cand : [M]  candidate label index들
    return: conf_cand [M]
    """
    conf = np.empty_like(cand, dtype=np.float32)
    for k, c in enumerate(cand):
        par = parents[c]
        sib = siblings[c]
        neigh_vals = []

        if len(par) > 0:
            neigh_vals.append(sims[par].max())
        if len(sib) > 0:
            neigh_vals.append(sims[sib].max())

        neigh_max = max(neigh_vals) if neigh_vals else 0.0
        conf[k] = float(sims[c] - neigh_max)
    return conf

def hierarchical_beam_similarity_avg(
    doc_vec: np.ndarray,
    label_emb: np.ndarray,
    adj_upper: np.ndarray,
    roots: list[int] = [0],       # 여러 루트
    beam: int = 5,
    per_parent: str | int = "l+2",
    tau: float = 0.35,
    eps: float = 1e-9,
    max_depth: int | None = None,
    normalize: bool = False,      # 필요하면 True로
):
    doc = np.asarray(doc_vec, dtype=np.float32)
    L = np.asarray(label_emb, dtype=np.float32)
    A = np.asarray(adj_upper).astype(bool)
    N, d = L.shape

    if normalize:
        doc = doc / (np.linalg.norm(doc) + eps)
        L = L / (np.linalg.norm(L, axis=1, keepdims=True) + eps)

    # 로컬 점수
    sims = L @ doc
    p = 1.0 / (1.0 + np.exp(-sims / max(tau, 1e-6)))

    children = [np.flatnonzero(A[i]) for i in range(N)]

    S = np.full(N, -np.inf, dtype=np.float32)
    K = np.full(N, -np.inf, dtype=np.float32)
    Llen = np.zeros(N, dtype=np.int32)

    roots = list(roots)
    for r in roots:
        S[r] = 0.0
        Llen[r] = 0
        K[r] = -np.inf

    levels = [roots[:]]
    cur = roots[:]
    level_id = 0

    while True:
        cand_best = {}
        if isinstance(per_parent, str) and per_parent.startswith("l+"):
            # "l+2", "l+4" 등 일반 처리
            offset = int(per_parent[2:])
            k_parent = level_id + offset
        else:
            k_parent = int(per_parent)

        for par in cur:
            ch = children[par]
            if ch.size == 0:
                continue
            if ch.size > k_parent:
                idx = np.argpartition(-sims[ch], k_parent - 1)[:k_parent]
                ch = ch[idx]
            for c in ch:
                S_c = S[par] + float(p[c])
                L_c = Llen[par] + 1
                K_c = S_c / (L_c + eps)
                if (c not in cand_best) or (K_c > cand_best[c][2]):
                    cand_best[c] = (S_c, L_c, K_c)

        if not cand_best:
            break

        kept = sorted(cand_best.items(), key=lambda x: x[1][2], reverse=True)[:min(beam, len(cand_best))]
        next_level = [i for i, _ in kept]
        for i, (Si, Li, Ki) in kept:
            S[i], Llen[i], K[i] = Si, Li, Ki

        levels.append(next_level)
        cur = next_level
        level_id += 1
        if max_depth is not None and level_id >= max_depth:
            break

    return K, levels, sims, p



def topk_labels_by_avg(
    doc_vec, label_emb, adj_upper, rootㄴ=(0,), beam=5, per_parent="l+2", k=5, **kw
):
    """평균 점수 기반 최종 상위 k 라벨(루트 제외)."""
    K, levels, sims, p = hierarchical_beam_similarity_avg(
        doc_vec, label_emb, adj_upper, root=list(roots), beam=beam, per_parent=per_parent, **kw
    )
    root_set = set(roots)
    order = np.argsort(-K)
    order = [i for i in order if i not in root_set and np.isfinite(K[i])]
    top = order[:k]
    return top, K[top]

In [22]:
"""
Self-training pipeline with hierarchical silver labeling and dynamic dataloaders.

- Reads document/label embeddings CSVs (first column "id", rest feat000..feat127)
- Reads upper-triangular adjacency (A[i,j]=1 means i->j)
- Makes initial silver labels via hierarchical beam search (average score)
- Splits into train/val on silver set; keeps the rest as unlabeled pool
- Trains a multi-label classifier (Linear/MLP) with BCEWithLogitsLoss
- Each epoch, pseudo-labels unlabeled docs whose predicted probs exceed a threshold
- Adds them to the training set (up to top_k per doc), with patience-based early stopping

Run example
-----------
python self_training_pipeline.py \
  --doc_csv docs.csv \
  --label_csv labels.csv \
  --adj adj.npy \
  --val_ratio 0.2 --epochs 50 --patience 5 \
  --silver_threshold 0.60 --silver_topk 3 --beam 5 --tau 0.35 --root_id 0 \
  --pseudo_threshold 0.70 --pseudo_topk 3 --batch_size 256 --lr 1e-3
"""
from __future__ import annotations
import argparse
from dataclasses import dataclass
from pathlib import Path
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
def load_embeddings_csv(path: str | Path, id_col: str = "id") -> Tuple[List[int], np.ndarray]:
    """Load embeddings from CSV where the first column is an id and the rest are feature columns.
    Returns (ids, float32 matrix).
    """
    df = pd.read_csv(path)
    cols = list(df.columns)
    if id_col in df.columns:
        id_series = df[id_col]
        X = df.drop(columns=[id_col])
    else:
        # Fallback: use the first column as id
        id_series = df.iloc[:, 0]
        X = df.iloc[:, 1:]
    ids = id_series.astype(int).tolist()
    X = X.to_numpy(dtype=np.float32)
    return ids, X


# ----------------------------- Datasets -----------------------------

class MultiLabelDataset(Dataset):
    def __init__(self, X: np.ndarray, Y: np.ndarray, indices: List[int] | None = None):
        self.X = X
        self.Y = Y
        self.indices = np.array(indices if indices is not None else np.arange(X.shape[0]), dtype=np.int64)
    def __len__(self):
        return self.indices.shape[0]
    def __getitem__(self, idx: int):
        i = int(self.indices[idx])
        x = torch.from_numpy(self.X[i])
        y = torch.from_numpy(self.Y[i])
        return x, y

class UnlabeledDataset(Dataset):
    def __init__(self, X: np.ndarray, indices: List[int]):
        self.X = X
        self.indices = np.array(indices, dtype=np.int64)
    def __len__(self):
        return self.indices.shape[0]
    def __getitem__(self, idx: int):
        i = int(self.indices[idx])
        x = torch.from_numpy(self.X[i])
        return x, i

# ----------------------------- Model -----------------------------

class MLPHead(nn.Module):
    def __init__(self, in_dim: int, out_dim: int, hidden: int | None = 256, dropout: float = 0.1):
        super().__init__()
        if hidden is None or hidden <= 0:
            self.net = nn.Sequential(
                nn.LayerNorm(in_dim),
                nn.Linear(in_dim, out_dim),
            )
        else:
            self.net = nn.Sequential(
                nn.LayerNorm(in_dim),
                nn.Linear(in_dim, hidden),
                nn.GELU(),
                nn.Dropout(dropout),
                nn.Linear(hidden, out_dim),
            )
    def forward(self, x):
        return self.net(x)

# ----------------------------- Utils -----------------------------

def to_device(batch, device):
    if isinstance(batch, (tuple, list)):
        return [b.to(device) if torch.is_tensor(b) else b for b in batch]
    return batch.to(device)


def micro_f1(y_true: np.ndarray, y_prob: np.ndarray, thr: float = 0.5, eps: float = 1e-9) -> float:
    y_pred = (y_prob >= thr).astype(np.float32)
    tp = (y_true * y_pred).sum()
    fp = ((1 - y_true) * y_pred).sum()
    fn = (y_true * (1 - y_pred)).sum()
    prec = tp / (tp + fp + eps)
    rec = tp / (tp + fn + eps)
    f1 = 2 * prec * rec / (prec + rec + eps)
    return float(f1)

# -------- Initial silver labeling (no CSV save; in-memory) --------

# --------------------------------
# K-beam + confidence 기반 silver 생성
# --------------------------------
def make_initial_silver(
    docs: np.ndarray,         # [N_docs, d_doc]
    label_emb: np.ndarray,    # [C, d_lab]
    adj_upper: np.ndarray,    # [C, C]  parent -> child (upper adj)
    roots: List[int],
    silver_topk: int = 3,
    beam: int = 15,
    per_parent = "l+4",
    tau: float = 0.35,
    min_conf: float = 0.0,    # conf가 이 값보다 작으면 버림
    eps: float = 1e-9,
) -> List[List[int]]:
    """
    - hierarchical_beam_similarity_avg + conf(D,c) + median 필터 기반 silver 생성
    - conf(D,c) = sim(D,c) - max_{par/sib} sim(D, c')
    - conf(D,c) >= median_c && conf(D,c) >= min_conf 인 라벨만 core 후보로 남김
    """
    N_docs = docs.shape[0]
    C = label_emb.shape[0]

    parents, children = build_parents_children(adj_upper)
    siblings = build_siblings(parents, children)

    # 라벨별 conf 값 모으기
    conf_per_label: list[list[float]] = [[] for _ in range(C)]

    # 문서별 candidate / conf 저장 (두 번째 pass에 사용)
    cand_per_doc: list[np.ndarray] = []
    conf_per_doc: list[np.ndarray] = []

    # ---------- 1st pass: K-beam + conf(D,c) 수집 ----------
    for d_idx in range(N_docs):
        doc_vec = docs[d_idx]

        # K-beam으로 후보 라벨 탐색
        K, levels, sims, p = hierarchical_beam_similarity_avg(
            doc_vec,
            label_emb,
            adj_upper,
            roots=roots,
            beam=beam,
            per_parent=per_parent,
            tau=tau,
            eps=eps,
            max_depth=None,
            normalize=False,
        )

        # beam에서 방문된 라벨들을 candidate로
        cand = sorted({int(c) for lvl in levels for c in lvl})
        cand = np.array(cand, dtype=np.int64)

        if cand.size == 0:
            cand_per_doc.append(cand)
            conf_per_doc.append(np.zeros(0, dtype=np.float32))
            continue

        sims_doc = sims  # [C]
        conf_cand = compute_conf_doc(sims_doc, cand, parents, siblings)

        # 라벨별 conf 목록에 추가
        for c, cf in zip(cand, conf_cand):
            conf_per_label[c].append(float(cf))

        cand_per_doc.append(cand)
        conf_per_doc.append(conf_cand)

    # ---------- 라벨별 median(conf) 계산 ----------
    median_conf = np.full(C, np.inf, dtype=np.float32)
    for c in range(C):
        vals = conf_per_label[c]
        if len(vals) == 0:
            continue
        median_conf[c] = float(np.median(vals))

    # ---------- 2nd pass: median + min_conf 기준으로 silver 확정 ----------
    silver: List[List[int]] = []
    for cand, conf_cand in zip(cand_per_doc, conf_per_doc):
        if cand.size == 0:
            silver.append([])
            continue

        kept = []
        for c, cf in zip(cand, conf_cand):
            if not np.isfinite(cf):
                continue
            if cf < median_conf[c]:
                continue
            if cf < min_conf:
                continue
            kept.append((int(c), float(cf)))

        kept.sort(key=lambda x: x[1], reverse=True)
        selected = [c for c, _ in kept[:silver_topk]]
        silver.append(selected)

    return silver

def make_initial_silver_hier_conf(
    docs: np.ndarray,         # [N_docs, d_doc]
    label_emb: np.ndarray,    # [C, d_lab]
    adj_upper: np.ndarray,    # [C, C]  parent -> child
    roots: List[int],
    silver_topk: int = 3,
    beam: int = 15,
    per_parent = "l+4",
    tau: float = 0.35,
    silver_threshold: float = 0.65,  # 🔴 p-threshold
    min_conf: float = 0.0,           # 🔴 conf-threshold
    eps: float = 1e-9,
) -> List[List[int]]:
    """
    - hierarchical_beam_similarity_avg + conf(D,c) + median + p-threshold 기반 silver 생성
    - candidate 조건:
        1) beam search 통해 방문된 노드
        2) p(D,c) >= silver_threshold
    - 최종 core 조건:
        1) conf(D,c) >= median_c
        2) conf(D,c) >= min_conf
    """
    N_docs = docs.shape[0]
    C = label_emb.shape[0]

    parents, children = build_parents_children(adj_upper)
    siblings = build_siblings(parents, children)

    # 라벨별 conf 값 모으기
    conf_per_label: list[list[float]] = [[] for _ in range(C)]

    # 문서별 candidate / conf 저장
    cand_per_doc: list[np.ndarray] = []
    conf_per_doc: list[np.ndarray] = []

    # ---------- 1st pass: K-beam + (p, conf) 수집 ----------
    for d_idx in range(N_docs):
        doc_vec = docs[d_idx]

        K, levels, sims, p = hierarchical_beam_similarity_avg(
            doc_vec,
            label_emb,
            adj_upper,
            roots=roots,
            beam=beam,
            per_parent=per_parent,
            tau=tau,
            eps=eps,
            max_depth=None,
            normalize=False,
        )

        # beam에서 방문된 라벨들을 candidate로
        cand = sorted({int(c) for lvl in levels for c in lvl})
        cand = np.array(cand, dtype=np.int64)

        if cand.size == 0:
            cand_per_doc.append(cand)
            conf_per_doc.append(np.zeros(0, dtype=np.float32))
            continue

        sims_doc = sims        # [C]
        p_doc    = p           # [C]

        # 🔴 p-threshold 적용: p >= silver_threshold 인 것만 남김
        mask_thr = p_doc[cand] >= silver_threshold
        cand = cand[mask_thr]

        if cand.size == 0:
            cand_per_doc.append(cand)
            conf_per_doc.append(np.zeros(0, dtype=np.float32))
            continue

        conf_cand = compute_conf_doc(sims_doc, cand, parents, siblings)

        # 라벨별 conf 목록에 추가
        for c, cf in zip(cand, conf_cand):
            conf_per_label[c].append(float(cf))

        cand_per_doc.append(cand)
        conf_per_doc.append(conf_cand)

    # ---------- 라벨별 median(conf) 계산 ----------
    median_conf = np.full(C, np.inf, dtype=np.float32)
    for c in range(C):
        vals = conf_per_label[c]
        if len(vals) == 0:
            continue
        median_conf[c] = float(np.median(vals))

    # ---------- 2nd pass: median + min_conf 기준으로 silver 확정 ----------
    silver: List[List[int]] = []
    for cand, conf_cand in zip(cand_per_doc, conf_per_doc):
        if cand.size == 0:
            silver.append([])
            continue

        kept = []
        for c, cf in zip(cand, conf_cand):
            if not np.isfinite(cf):
                continue
            if cf < median_conf[c]:  # median 조건
                continue
            if cf < min_conf:        # 절대 threshold 조건
                continue
            kept.append((int(c), float(cf)))

        kept.sort(key=lambda x: x[1], reverse=True)
        selected = [c for c, _ in kept[:silver_topk]]
        silver.append(selected)

    return silver
# ------------------------ Train / Self-Training ------------------------

def train_epoch(model, loader, optim, device, criterion):
    model.train()
    total = 0.0
    for x, y in loader:
        x, y = to_device(x, device), to_device(y, device)
        optim.zero_grad(set_to_none=True)
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optim.step()
        total += float(loss.detach().cpu().item()) * x.size(0)
    return total / max(1, len(loader.dataset))


def eval_epoch(model, loader, device, criterion, thr=0.5):
    model.eval()
    total = 0.0
    ys = []
    ps = []
    with torch.no_grad():
        for x, y in loader:
            x, y = to_device(x, device), to_device(y, device)
            logits = model(x)
            loss = criterion(logits, y)
            total += float(loss.detach().cpu().item()) * x.size(0)
            prob = torch.sigmoid(logits).detach().cpu().numpy()
            ys.append(y.detach().cpu().numpy())
            ps.append(prob)
    y_true = np.concatenate(ys, axis=0)
    y_prob = np.concatenate(ps, axis=0)
    f1 = micro_f1(y_true, y_prob, thr=thr)
    return total / max(1, len(loader.dataset)), f1, y_prob

def _compute_conf_for_doc(prob_row: np.ndarray,
                          cand: np.ndarray,
                          parents,
                          siblings) -> np.ndarray:
    """
    prob_row: [C]  (p(D, :))
    cand: [M] candidate label index
    return: conf_cand [M]
    """
    conf = np.empty(len(cand), dtype=np.float32)
    for k, c in enumerate(cand):
        par = parents[c]
        sib = siblings[c]

        neigh_vals = []
        if len(par) > 0:
            neigh_vals.append(prob_row[par].max())
        if len(sib) > 0:
            neigh_vals.append(prob_row[sib].max())

        neigh_max = max(neigh_vals) if neigh_vals else 0.0
        conf[k] = float(prob_row[c] - neigh_max)
    return conf





In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader



doc_ids = np.arange(len(doc_embeddings), dtype=np.int64)   # 0..num_docs-1
X = doc_embeddings.astype(np.float32)                      # [num_docs, d_doc]

# 라벨 임베딩 세팅
label_ids = np.arange(len(label_keys), dtype=np.int64)     # 0..530
L = np.vstack([label_embeddings[k] for k in label_keys]).astype(np.float32)   # [531, d_label]


# 1) 라벨 순서와 B(부모->자식) 맞추기
order = np.argsort(label_ids)
label_ids = [label_ids[i] for i in order]
L = L[order]
assert B.shape == (L.shape[0], L.shape[0]), "Adjacency/label size mismatch"

# 2) 계층 silver 만들기
silver = make_initial_silver_hier_conf(
    X,               # 문서 임베딩
    L,               # 라벨 임베딩 (TF-IDF든 GAT든)
    B,               # parent->child adj
    roots=roots,
    silver_topk=3,
    beam=15,
    per_parent="l+4",
    tau=0.35,
    silver_threshold=0.6,  # p(D,c) threshold
    min_conf=0.0,           # conf threshold (필요하면 0.05 같은 값으로 올려도 됨)
)

# -------------------------------------------------
# 1) 계층 정보에서 parents / children 뽑기
#    B[parent, child] = 1 이라고 했으니까 그대로 씀
# -------------------------------------------------
# B: [C, C] (parent -> child)
def build_parents_children(adj):
    C = adj.shape[0]
    parents = [np.flatnonzero(adj[:, j]).astype(np.int64) for j in range(C)]
    children = [np.flatnonzero(adj[j]).astype(np.int64) for j in range(C)]
    return parents, children

parents, children = build_parents_children(B)


# -------------------------------------------------
# 2) silver → 계층 pos/neg 마스크로 변환
# -------------------------------------------------
def build_pos_neg_masks(silver, parents, children, num_labels):
    """
    silver: list[list[int]]  # 문서마다 core label index들
    parents / children: list[np.ndarray]
    return:
      pos_masks: np.array [N_docs, C]
      neg_masks: np.array [N_docs, C]
    """
    N = len(silver)
    C = num_labels
    pos_masks = np.zeros((N, C), dtype=np.float32)
    neg_masks = np.zeros((N, C), dtype=np.float32)

    all_idx = np.arange(C)

    for i, core in enumerate(silver):
        core = list(core)
        # 1) core의 부모까지 positive
        pos_set = set(core)
        for c in core:
            for p in parents[c]:
                pos_set.add(int(p))

        # 2) children은 나중에 negative에서 제외
        child_set = set()
        for c in core:
            for ch in children[c]:
                child_set.add(int(ch))

        # pos 마스크
        for p in pos_set:
            pos_masks[i, p] = 1.0

        # neg = 전체 - pos - children
        for j in all_idx:
            if j in pos_set:
                continue
            if j in child_set:
                continue
            neg_masks[i, j] = 1.0

    return pos_masks, neg_masks

# -------------------------------------------------
# 3) Dataset: 문서 임베딩 + pos/neg 마스크
# -------------------------------------------------
class HierMultiLabelDataset(Dataset):
    def __init__(self, X, pos_masks, neg_masks, indices=None):
        self.X = X.astype(np.float32)
        self.pos = pos_masks.astype(np.float32)
        self.neg = neg_masks.astype(np.float32)
        if indices is None:
            self.indices = np.arange(self.X.shape[0], dtype=np.int64)
        else:
            self.indices = np.array(indices, dtype=np.int64)

    def __len__(self):
        return self.indices.shape[0]

    def __getitem__(self, idx):
        i = int(self.indices[idx])
        x = torch.from_numpy(self.X[i])
        pos = torch.from_numpy(self.pos[i])
        neg = torch.from_numpy(self.neg[i])
        return x, pos, neg

class UnlabeledDataset(Dataset):
    def __init__(self, X, indices):
        self.X = X.astype(np.float32)
        self.indices = np.array(indices, dtype=np.int64)
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        i = int(self.indices[idx])
        return torch.from_numpy(self.X[i]), i


# -------------------------------------------------
# 4) Bilinear classifier
#    doc_emb: [B, d_doc]
#    label_emb: [C, d_lab]  (미리 GAT로 만든 거)
#    점수: doc @ W @ label_emb^T
# -------------------------------------------------
class BilinearHierClassifier(nn.Module):
    def __init__(self, doc_dim, label_emb, hidden_dim=None):
        super().__init__()
        # label_emb는 파라미터로 들고있되, 업데이트 안 한다고 가정(원하면 nn.Parameter로)
        self.register_buffer("label_emb", torch.tensor(label_emb, dtype=torch.float32))
        C, d_lab = self.label_emb.shape
        self.doc_dim = doc_dim
        self.label_dim = d_lab

        if hidden_dim is None:
            # 바로 doc_dim -> label_dim
            self.interaction = nn.Linear(doc_dim, d_lab, bias=False)
            self.proj = None
        else:
            # doc_dim -> hidden -> label_dim 같은 것도 가능
            self.interaction = nn.Sequential(
                nn.Linear(doc_dim, hidden_dim),
                nn.GELU(),
                nn.Linear(hidden_dim, d_lab, bias=False),
            )

    def forward(self, x):
        """
        x: [B, d_doc]
        return: logits [B, C]
        """
        # x -> same dim as label
        h = self.interaction(x)                             # [B, d_lab]
        # [B, d_lab] @ [d_lab, C] -> [B, C]
        logits = torch.matmul(h, self.label_emb.t())
        return logits

# -------------------------------------------------
# 5) loss: 계층 마스크를 씌운 BCE
# -------------------------------------------------
def hierarchical_bce_loss(logits, pos_mask, neg_mask):
    # logits: [B, C]
    # pos_mask, neg_mask: [B, C]
    loss_pos = -(pos_mask * F.logsigmoid(logits)).sum()
    loss_neg = -(neg_mask * F.logsigmoid(-logits)).sum()
    denom = (pos_mask.sum() + neg_mask.sum()).clamp(min=1.0)
    return (loss_pos + loss_neg) / denom

# -------------------------------------------------
# 6) 학습 루프 예시
# -------------------------------------------------
# 이미 있는 것들: X (문서 BERT 임베딩) : [N_docs, d_doc]
#                  L (라벨 GAT 임베딩)  : [C, d_lab]
#                  B_adj (부모->자식)   : [C, C]
#                  silver (list[list[int]]) : 문서별 core label index
def train_epoch_hier(model, loader, opt, device):
    model.train()
    total = 0.0
    for xb, posb, negb in loader:
        xb = xb.to(device)
        posb = posb.to(device)
        negb = negb.to(device)
        logits = model(xb)
        loss = hierarchical_bce_loss(logits, posb, negb)
        opt.zero_grad()
        loss.backward()
        opt.step()
        total += loss.item() * xb.size(0)
    return total / len(loader.dataset)

# 1) micro F1 계산
def micro_f1_from_logits(logits, pos_mask, thr=0.5, eps=1e-9):
    """
    logits: [B, C]
    pos_mask: [B, C]  (1: positive, 0: else)
    """
    probs = torch.sigmoid(logits)
    preds = (probs >= thr).float()

    y_true = pos_mask
    y_pred = preds

    tp = (y_true * y_pred).sum()
    fp = ((1 - y_true) * y_pred).sum()
    fn = (y_true * (1 - y_pred)).sum()

    precision = tp / (tp + fp + eps)
    recall    = tp / (tp + fn + eps)
    f1 = 2 * precision * recall / (precision + recall + eps)
    return f1.item()

# 2) eval 함수 수정: loss + f1 둘 다
def eval_epoch_hier(model, loader, device, k=3, thr=None):
    model.eval()
    total_loss = 0.0
    f1_list = []
    with torch.no_grad():
        for xb, posb, negb in loader:
            xb = xb.to(device)
            posb = posb.to(device)
            negb = negb.to(device)

            logits = model(xb)
            loss = hierarchical_bce_loss(logits, posb, negb)  # 위에 바꾼 버전
            total_loss += loss.item() * xb.size(0)

            probs = torch.sigmoid(logits)

            if thr is not None:
                pred = (probs >= thr).float()
            else:
                # top-k 방식
                B, C = probs.shape
                pred = torch.zeros_like(probs)
                topk = probs.topk(k, dim=1).indices
                pred.scatter_(1, topk, 1.0)

            # micro-f1
            y_true = posb
            y_pred = pred
            tp = (y_true * y_pred).sum().item()
            fp = ((1 - y_true) * y_pred).sum().item()
            fn = (y_true * (1 - y_pred)).sum().item()
            prec = tp / (tp + fp + 1e-9)
            rec  = tp / (tp + fn + 1e-9)
            f1   = 2 * prec * rec / (prec + rec + 1e-9)
            f1_list.append(f1)

    avg_loss = total_loss / len(loader.dataset)
    avg_f1 = float(np.mean(f1_list)) if f1_list else 0.0
    return avg_loss, avg_f1
def pseudo_label_and_grow_hier_conf(
    model,
    unl_ds,             # UnlabeledDataset
    parents,
    children,
    num_labels,
    device,
    pseudo_threshold: float = 0.45,   # p(D,c) threshold
    pseudo_topk: int = 3,
    batch_size: int = 512,
    max_candidates: int = 32,         # 한 문서당 최대 후보 라벨 수
):
    """
    - unl_ds에서 pseudo-label로 쓸 문서를 뽑고
    - confidence + label-wise median 기반으로 core 라벨 결정
    - core + 부모 = pos, 자식 = unknown, 나머지 = neg
    """
    if len(unl_ds) == 0:
        return [], None, None

    loader = DataLoader(unl_ds, batch_size=batch_size, shuffle=False)
    model.eval()

    C = num_labels
    siblings = build_siblings(parents, children)

    # 1-pass: 후보 라벨, conf, prob 수집 + 라벨별 conf 분포 저장
    conf_per_label = [[] for _ in range(C)]

    doc_indices = []         # 각 entry: i_doc (원본 인덱스)
    cand_list = []           # 각 entry: np.array [M_doc]
    conf_cand_list = []      # 각 entry: np.array [M_doc]
    prob_cand_list = []      # 각 entry: np.array [M_doc]

    with torch.no_grad():
        for xb, idxs in loader:
            xb = xb.to(device)
            logits = model(xb)
            prob = torch.sigmoid(logits).cpu().numpy()   # [B, C]
            idxs_np = idxs.cpu().numpy().tolist()

            for p_row, i_doc in zip(prob, idxs_np):
                # 확률 내림차순
                order = np.argsort(-p_row)

                # p >= pseudo_threshold인 애들 중 상위 max_candidates만 후보
                cand = [j for j in order[:max_candidates] if p_row[j] >= pseudo_threshold]
                if len(cand) == 0:
                    continue

                cand = np.array(cand, dtype=np.int64)
                conf_cand = _compute_conf_for_doc(p_row, cand, parents, siblings)
                prob_cand = p_row[cand].astype(np.float32)

                # 라벨별 conf 분포에 추가
                for c, cf in zip(cand, conf_cand):
                    conf_per_label[c].append(float(cf))

                doc_indices.append(int(i_doc))
                cand_list.append(cand)
                conf_cand_list.append(conf_cand)
                prob_cand_list.append(prob_cand)

    if not doc_indices:
        # 이번 epoch에는 아무것도 pseudo-label로 안 넣음
        return [], None, None

    # 라벨별 median(conf) 계산
    median_conf = np.full(C, -np.inf, dtype=np.float32)
    for c in range(C):
        vals = conf_per_label[c]
        if len(vals) == 0:
            continue
        median_conf[c] = float(np.median(vals))

    # 2-pass: median 기반으로 실제 core 라벨 선택 + pos/neg 마스크 생성
    new_idx = []
    new_pos_list = []
    new_neg_list = []

    for i_doc, cand, conf_cand, prob_cand in zip(
        doc_indices, cand_list, conf_cand_list, prob_cand_list
    ):
        keep = []
        for c, cf, pc in zip(cand, conf_cand, prob_cand):
            # conf가 라벨별 median 이상일 때만 채택
            if cf < median_conf[c]:
                continue
            keep.append((int(c), float(pc), float(cf)))

        if not keep:
            continue

        # 확률 기준으로 정렬 후 top-k
        keep.sort(key=lambda x: x[1], reverse=True)  # x[1] = prob
        core = [c for c, _, _ in keep[:pseudo_topk]]

        if len(core) == 0:
            continue

        # 계층 pos/neg 구성 (기존 함수와 동일 로직)
        pos = set(core)
        for c in core:
            for pa in parents[c]:
                pos.add(int(pa))

        child = set()
        for c in core:
            for ch in children[c]:
                child.add(int(ch))

        pos_mask = np.zeros(C, dtype=np.float32)
        neg_mask = np.zeros(C, dtype=np.float32)

        for j in pos:
            pos_mask[j] = 1.0
        for j in range(C):
            if j in pos:
                continue
            if j in child:
                continue
            neg_mask[j] = 1.0

        new_idx.append(int(i_doc))
        new_pos_list.append(pos_mask)
        new_neg_list.append(neg_mask)

    if len(new_idx) == 0:
        return [], None, None

    new_pos = np.stack(new_pos_list, axis=0)
    new_neg = np.stack(new_neg_list, axis=0)
    return new_idx, new_pos, new_neg



device = "cuda" if torch.cuda.is_available() else "cpu"
has_silver = np.array([len(lbls) > 0 for lbls in silver], dtype=bool)
N_docs = X.shape[0]
C = L.shape[0]

# silver 있는 문서 / 없는 문서
has_silver = np.array([len(lbls) > 0 for lbls in silver], dtype=bool)
idx_silver = np.flatnonzero(has_silver)      # 여기가 train/val 후보
idx_unl    = np.flatnonzero(~has_silver)     # 진짜 unl

print("total:", N_docs)
print("with silver:", len(idx_silver))
print("unlabeled :", len(idx_unl))

# 이제 train/val은 silver 있는 애들만 섞어서 나눈다
rng = np.random.default_rng(42)
rng.shuffle(idx_silver)
n_val = int(len(idx_silver) * 0.2)
idx_val   = idx_silver[:n_val]
idx_train = idx_silver[n_val:]

# parents, children 만들기
def build_parents_children(adj):
    C = adj.shape[0]
    parents = [np.flatnonzero(adj[:, j]).astype(np.int64) for j in range(C)]
    children = [np.flatnonzero(adj[j]).astype(np.int64) for j in range(C)]
    return parents, children

parents, children = build_parents_children(B)

pos_masks = np.zeros((N_docs, C), dtype=np.float32)
neg_masks = np.zeros((N_docs, C), dtype=np.float32)

for i in idx_silver:  # silver 있는 애만 돈다
    core = silver[i]

    # 1) core + parents
    pos = set(core)
    for c in core:
        for p in parents[c]:
            pos.add(int(p))

    # 2) children은 모름
    child = set()
    for c in core:
        for ch in children[c]:
            child.add(int(ch))

    for p in pos:
        pos_masks[i, p] = 1.0

    for j in range(C):
        if j in pos:      # 이미 양성
            continue
        if j in child:    # 모름
            continue
        neg_masks[i, j] = 1.0



train_ds = HierMultiLabelDataset(X, pos_masks, neg_masks, indices=idx_train)
val_ds   = HierMultiLabelDataset(X, pos_masks, neg_masks, indices=idx_val) if len(idx_val) > 0 else None
unl_ds   = UnlabeledDataset(X, idx_unl.tolist())
print(len(train_ds),len(val_ds),len(unl_ds))

train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=256, shuffle=False)

model = BilinearHierClassifier(doc_dim=X.shape[1], label_emb=L, hidden_dim=256).to(device)

opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
epochs = 500

N_labels = L.shape[0]
best_f1 = -1.0
patience = 20
no_improve = 0
warmup_self = 1   # 1 epoch은 self-training 안 하게 해서 한 번 안정화

for epoch in range(1, epochs + 1):
    # train
    tr_loss = train_epoch_hier(model, train_loader, opt, device)

    # val: f1 기준
    if val_loader is not None and len(val_ds) > 0:
        va_loss, va_f1 = eval_epoch_hier(model, val_loader, device, k=3)
        print(f"Epoch {epoch:03d} | train_loss={tr_loss:.3f}  val_loss={va_loss:.3f}  val_f1={va_f1:.3f}")

        # early stopping을 f1로
        if va_f1 > best_f1 + 1e-6:
            best_f1 = va_f1
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"Early stopping at epoch {epoch} (best f1={best_f1:.4f})")
                break
    else:
        print(f"Epoch {epoch:03d} | train_loss={tr_loss:.3f}")

    # self-training: 1에폭에 전부 들어가는 거 방지용으로 warmup 넣음
    if epoch <= warmup_self:
        print("  + (skip pseudo-labeling on warmup epoch)")
        continue

    new_idx, new_pos, new_neg = pseudo_label_and_grow_hier_conf(
        model,
        unl_ds,
        parents,
        children,
        C,
        device=device,
        pseudo_threshold=0.45,   # p(D,c) 기본 threshold
        pseudo_topk=3,
        batch_size=512,
        max_candidates=32,
    )

    if len(new_idx) > 0:
        # 전역 마스크 갱신
        pos_masks[new_idx] = new_pos
        neg_masks[new_idx] = new_neg

        # unl에서 제거
        keep_mask = ~np.isin(unl_ds.indices, np.array(new_idx, dtype=np.int64))
        unl_ds.indices = unl_ds.indices[keep_mask]

        # train에 추가
        train_ds.indices = np.concatenate([train_ds.indices, np.array(new_idx, dtype=np.int64)])
        train_loader = DataLoader(train_ds, batch_size=256, shuffle=True, drop_last=False)

        print(f"  + Added {len(new_idx)} pseudo-labeled docs (unl pool → {len(unl_ds)} left)")
    else:
        print("  + No pseudo-labeled docs added this epoch")


total: 29487
with silver: 12449
unlabeled : 17038
9960 2489 17038
Epoch 001 | train_loss=0.433  val_loss=0.103  val_f1=0.008
  + (skip pseudo-labeling on warmup epoch)
Epoch 002 | train_loss=0.052  val_loss=0.037  val_f1=0.080
  + No pseudo-labeled docs added this epoch
Epoch 003 | train_loss=0.034  val_loss=0.032  val_f1=0.127
  + No pseudo-labeled docs added this epoch
Epoch 004 | train_loss=0.031  val_loss=0.030  val_f1=0.135
  + No pseudo-labeled docs added this epoch
Epoch 005 | train_loss=0.030  val_loss=0.029  val_f1=0.135
  + No pseudo-labeled docs added this epoch
Epoch 006 | train_loss=0.029  val_loss=0.028  val_f1=0.135
  + No pseudo-labeled docs added this epoch
Epoch 007 | train_loss=0.028  val_loss=0.028  val_f1=0.135
  + No pseudo-labeled docs added this epoch
Epoch 008 | train_loss=0.028  val_loss=0.028  val_f1=0.135
  + No pseudo-labeled docs added this epoch
Epoch 009 | train_loss=0.028  val_loss=0.027  val_f1=0.135
  + No pseudo-labeled docs added this epoch
Epoch 01

In [26]:
import csv, os
from pathlib import Path
import numpy as np
import pandas as pd

# ------------ Paths (edit if needed) ------------
TEST_CORPUS = "Amazon_products/test/test_corpus.txt"   # lines: pid \t text
OUT_PATH    = "submission_bda.csv"
# ------------ Hyperparams ------------
MIN_LABS  = 2
MAX_LABS  = 3
BATCH = 1024
doc_ids, doc_texts = load_docs_txt(TEST_CORPUS)

# 2) 라벨 때 만든 vectorizer 재사용해서 임베딩 만들기
test_embeddings = build_doc_embeddings_from_existing_vectorizer(doc_texts, vectorizer)
test_embeddings = test_embeddings.astype(np.float32)   # [num_test, d]
# load test pids
pids = doc_ids   # 이미 문자열 id
if "L" in globals():
    if not isinstance(L, np.ndarray):
        # 예: L이 torch.Tensor인 경우
        L = L.detach().cpu().numpy().astype(np.float32)
else:
    raise ValueError("라벨 임베딩 L이 메모리에 없어! GAT 끝난 뒤의 임베딩을 L로 둬야 해.")

# 5) 라벨 id는 0..N-1로 생성 (네가 말한 대로 adjacency랑 순서가 이미 맞다고 했으니까)
lab_ids = np.arange(L.shape[0], dtype=np.int64)

# 6) adjacency도 메모리에 있는 걸 그대로 쓴다
#    여기서 A는 531x531 같은 numpy array라고 가정
assert B.shape == (L.shape[0], L.shape[0]), "Adjacency/label size mismatch"

# 7) children 리스트 미리 만들어두기
children = [np.flatnonzero(B[i]) for i in range(B.shape[0])]



In [27]:
model.eval()

def ancestors_of(node, adj):
    # adj[parent, child] = 1 가정
    parents = np.flatnonzero(adj[:, node])  # (N,)
    return parents.tolist()

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

IN_DIM = test_embeddings.shape[1]
missing = 0  # 지금은 쓸 일 없지만 원래 코드랑 형태 맞춰둠

with open(OUT_PATH, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["id", "label"])

    buf_x, buf_pid = [], []

    def flush():
        if not buf_x:
            return
        xb = torch.from_numpy(np.stack(buf_x, axis=0).astype(np.float32)).to(device)
        with torch.inference_mode():
            prob = torch.sigmoid(model(xb)).detach().cpu().numpy()
        prob = np.nan_to_num(prob, nan=-1.0, posinf=1.0, neginf=0.0)

        for pid, p in zip(buf_pid, prob):
            order = np.argsort(-p)

            # 1) 기본 후보 뽑기
            thr_keep = [i for i in order if p[i] >= 0.5][:MAX_LABS]
            if len(thr_keep) >= MIN_LABS:
                keep = thr_keep[:MAX_LABS]
            else:
                keep = order[:max(MIN_LABS, len(thr_keep))]
                if len(keep) < MIN_LABS:
                    keep = order[:MIN_LABS]

            # 2) 부모 후보
            parent_cands = []
            for c in keep:
                pars = ancestors_of(c, B)
                for pa in pars:
                    if pa not in keep and pa not in parent_cands:
                        parent_cands.append(pa)

            parent_cands.sort(key=lambda idx: p[idx], reverse=True)

            # 3) 남는 슬롯 부모로 채우기
            final_idxs = list(keep)
            for pa in parent_cands:
                if len(final_idxs) >= MAX_LABS:
                    break
                final_idxs.append(pa)

            # 4) 그래도 모자라면 확률순
            if len(final_idxs) < MIN_LABS:
                for idx in order:
                    if idx not in final_idxs:
                        final_idxs.append(idx)
                    if len(final_idxs) >= MIN_LABS:
                        break

            labels = sorted(int(lab_ids[i]) for i in final_idxs)
            w.writerow([pid, ",".join(map(str, labels))])

        buf_x.clear()
        buf_pid.clear()

    # 여기서 바로 pids와 test_embeddings를 같이 순회
    for pid, emb in zip(pids, test_embeddings):
        x = emb
        if x.dtype != np.float32:
            x = x.astype(np.float32, copy=False)
        buf_x.append(x)
        buf_pid.append(pid)
        if len(buf_x) >= BATCH:
            flush()
    flush()

print(f"Saved: {OUT_PATH} | samples={len(pids)} | min-max labels per sample={MIN_LABS}-{MAX_LABS} | missing_pids={missing}")


Saved: submission_bda.csv | samples=19658 | min-max labels per sample=2-3 | missing_pids=0


In [29]:
def build_parents_children(adj: np.ndarray):
    C = adj.shape[0]
    parents = [np.flatnonzero(adj[:, j]).astype(np.int64) for j in range(C)]
    children = [np.flatnonzero(adj[j]).astype(np.int64) for j in range(C)]
    return parents, children

def build_siblings(parents, children):
    C = len(parents)
    sibs = [set() for _ in range(C)]
    for c in range(C):
        for p in parents[c]:
            for ch in children[p]:
                if ch != c:
                    sibs[c].add(int(ch))
    sibs = [np.array(sorted(s), dtype=np.int64) for s in sibs]
    return sibs

def compute_conf_all(p: np.ndarray, parents, siblings) -> np.ndarray:
    C = p.shape[0]
    conf = np.zeros(C, dtype=np.float32)
    for c in range(C):
        neigh = []
        if len(parents[c]) > 0:
            neigh.append(p[parents[c]].max())
        if len(siblings[c]) > 0:
            neigh.append(p[siblings[c]].max())
        neigh_max = max(neigh) if neigh else 0.0
        conf[c] = float(p[c] - neigh_max)
    return conf


In [30]:
def select_labels_hierarchical(
    p: np.ndarray,          # [C] 확률
    B: np.ndarray,          # [C, C] parent -> child
    MIN_LABS: int,
    MAX_LABS: int,
    p_threshold: float = 0.5,
    conf_margin: float = 0.0,   # conf >= 이 값일 때만 core
):
    parents, children = build_parents_children(B)
    siblings = build_siblings(parents, children)

    C = p.shape[0]
    conf = compute_conf_all(p, parents, siblings)

    order = np.argsort(-p)  # 확률 내림차순 인덱스

    core = []
    for c in order:
        # 확률이 너무 낮고 이미 MIN_LABS 이상이면 중단
        if p[c] < p_threshold and len(core) >= MIN_LABS:
            break

        # 기본 확률 조건
        if p[c] < p_threshold:
            continue

        # confidence margin 조건
        if conf[c] < conf_margin:
            continue

        core.append(int(c))
        if len(core) >= MAX_LABS:
            break

    # 아무것도 못 뽑았으면 fallback: flat top-N
    if len(core) == 0:
        core = [int(i) for i in order[:MIN_LABS]]

    # core에 대해 조상(부모) 확장
    final = set(core)
    for c in core:
        for pa in ancestors_of(c, B):
            final.add(int(pa))
            if len(final) >= MAX_LABS:
                break

    # 그래도 MIN_LABS보다 적으면 flat 확률 높은 순으로 채우기
    if len(final) < MIN_LABS:
        for idx in order:
            if idx not in final:
                final.add(int(idx))
            if len(final) >= MIN_LABS:
                break

    # 최종 결과를 확률 내림차순으로 정렬해서 MAX_LABS만 유지
    final_list = list(final)
    final_list.sort(key=lambda idx: -p[idx])
    return final_list[:MAX_LABS]


In [31]:
model.eval()

def ancestors_of(node, adj):
    # adj[parent, child] = 1 가정
    parents = np.flatnonzero(adj[:, node])  # (N,)
    return parents.tolist()

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

IN_DIM = test_embeddings.shape[1]
missing = 0  # 지금은 쓸 일 없지만 원래 코드랑 형태 맞춰둠

with open(OUT_PATH, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["id", "label"])

    buf_x, buf_pid = [], []

    def flush():
        if not buf_x:
            return
        xb = torch.from_numpy(np.stack(buf_x, axis=0).astype(np.float32)).to(device)
        with torch.inference_mode():
            prob = torch.sigmoid(model(xb)).detach().cpu().numpy()
        prob = np.nan_to_num(prob, nan=-1.0, posinf=1.0, neginf=0.0)


        for pid, p in zip(buf_pid, prob):
            final_idxs = select_labels_hierarchical(
                p,
                B,
                MIN_LABS=MIN_LABS,
                MAX_LABS=MAX_LABS,
                p_threshold=0.5,     # 기존 threshold와 맞춰도 되고 조금 낮춰도 됨
                conf_margin=0.0,     # 처음엔 0.0로 시작, 나중에 0.05 등 튜닝
            )
        
            labels = sorted(int(lab_ids[i]) for i in final_idxs)
            w.writerow([pid, ",".join(map(str, labels))])
        buf_x.clear()
        buf_pid.clear()

    # 여기서 바로 pids와 test_embeddings를 같이 순회
    for pid, emb in zip(pids, test_embeddings):
        x = emb
        if x.dtype != np.float32:
            x = x.astype(np.float32, copy=False)
        buf_x.append(x)
        buf_pid.append(pid)
        if len(buf_x) >= BATCH:
            flush()
    flush()

print(f"Saved: {OUT_PATH} | samples={len(pids)} | min-max labels per sample={MIN_LABS}-{MAX_LABS} | missing_pids={missing}")


Saved: submission_bda.csv | samples=19658 | min-max labels per sample=2-3 | missing_pids=0
